In [ ]:
import pandas as pd
import os
import json
import csv

In [ ]:
data_path = "detekcia-edge-on-galaxii-classifications.csv"
data = pd.read_csv(data_path)
# os.makedirs("txt", exist_ok=True)
destination = f"{os.getcwd()}/yolo/galaxies/labels"

In [ ]:
null_size_data = []

for i in range(len(data)):
    tasks = json.loads(data.iloc[i, 11])
    is_tle = tasks[0]["value"]
    if is_tle != "Áno":
        continue

    annotations = tasks[1]["value"]
    subject_data = json.loads(data.iloc[i, 12])
    metadata = json.loads(data.iloc[i, 10])
    subject_id = str(data.iloc[i,13])
    try:
        image_name = subject_data[subject_id]["Filename"].split(".")[0]
        image_data = metadata["subject_dimensions"][0]
        image_width = image_data["naturalWidth"]
        image_height = image_data["naturalHeight"]
    except TypeError:
        null_size_data.append(image_name)

    with open(os.path.join(destination, f"{image_name}.txt"), mode='w') as txt_file:
        for annotation in annotations:
            x = annotation["x"]
            y = annotation["y"]
            width = annotation["width"]
            height = annotation["height"]

            fieldnames1 = [0, round((x+(width/2))/image_width, 5), round((y+(height/2))/image_height, 5), round(width/image_width, 5), round(height/image_height,5)]

            writer = csv.DictWriter(txt_file, fieldnames=fieldnames1, delimiter=' ')
            writer.writeheader()
            print(f"Just made normalized txt file for {image_name}")

print(f"Corrupted size data: {len(null_size_data)}")
print(null_size_data)